In [1]:
import torch
import torch.nn as nn
import timm 

import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import transforms

import sys 
sys.path.append("/homeB/liangxiaoyu/23w0322/vit_explain/jacobgil_vit_geadcam/grad_cam/")
#https://zhuanlan.zhihu.com/p/64893308
from utils import GradCAM, show_cam_on_image, center_crop_img

model = timm.create_model("vit_small_patch16_224")
model

/homeB/liangxiaoyu/anaconda3/envs/p3.9g/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=False)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=2304, bias=True)
        (act): GELU(approximate=none)
        (fc2): Linear(in_features=2304, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): Att

timm 模型      def reset_classifier(self, num_classes, global_pool=''):
        self.num_classes = num_classes
        self.head = nn.Linear(self.embed_dim, num_classes) if num_classes > 0 else nn.Identity()

    def forward_features(self, x):
        B = x.shape[0]
        x = self.patch_embed(x)

        cls_tokens = self.cls_token.expand(B, -1, -1)  # stole cls_tokens impl from Phil Wang, thanks
        x = torch.cat((cls_tokens, x), dim=1)
        x = x + self.pos_embed
        x = self.pos_drop(x)

        for blk in self.blocks:
            x = blk(x)

        x = self.norm(x)
        return x[:, 0]

    def forward(self, x):
        x = self.forward_features(x)
        x = self.head(x)
        return x

        有head

这个人https://github.com/WZMIAOMIAO/deep-learning-for-image-processing/blob/master/pytorch_classification/grad_cam/vit_model.py

里面
class VisionTransformer(nn.Module):
    def __init__(self, img_size=224, patch_size=16, in_c=3, num_classes=1000,
                 embed_dim=768, depth=12, num_heads=12, mlp_ratio=4.0, qkv_bias=True,
                 qk_scale=None, representation_size=None, distilled=False, drop_ratio=0.,
                 attn_drop_ratio=0., drop_path_ratio=0., embed_layer=PatchEmbed, norm_layer=None,
                 act_layer=None):
        """
        Args:
            img_size (int, tuple): input image size
            patch_size (int, tuple): patch size
            in_c (int): number of input channels
            num_classes (int): number of classes for classification head
            embed_dim (int): embedding dimension
            depth (int): depth of transformer
            num_heads (int): number of attention heads
            mlp_ratio (int): ratio of mlp hidden dim to embedding dim
            qkv_bias (bool): enable bias for qkv if True
            qk_scale (float): override default qk scale of head_dim ** -0.5 if set
            representation_size (Optional[int]): enable and set representation layer (pre-logits) to this value if set
            distilled (bool): model includes a distillation token and head as in DeiT models
            drop_ratio (float): dropout rate
            attn_drop_ratio (float): attention dropout rate
            drop_path_ratio (float): stochastic depth rate
            embed_layer (nn.Module): patch embedding layer
            norm_layer: (nn.Module): normalization layer
        """
        super(VisionTransformer, self).__init__()
        self.num_classes = num_classes
        self.num_features = self.embed_dim = embed_dim  # num_features for consistency with other models
        self.num_tokens = 2 if distilled else 1
        norm_layer = norm_layer or partial(nn.LayerNorm, eps=1e-6)
        act_layer = act_layer or nn.GELU

        self.patch_embed = embed_layer(img_size=img_size, patch_size=patch_size, in_c=in_c, embed_dim=embed_dim)
        num_patches = self.patch_embed.num_patches

        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.dist_token = nn.Parameter(torch.zeros(1, 1, embed_dim)) if distilled else None
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + self.num_tokens, embed_dim))
        self.pos_drop = nn.Dropout(p=drop_ratio)

        dpr = [x.item() for x in torch.linspace(0, drop_path_ratio, depth)]  # stochastic depth decay rule
        self.blocks = nn.Sequential(*[
            Block(dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                  drop_ratio=drop_ratio, attn_drop_ratio=attn_drop_ratio, drop_path_ratio=dpr[i],
                  norm_layer=norm_layer, act_layer=act_layer)
            for i in range(depth)
        ])
        self.norm = norm_layer(embed_dim)

        # Representation layer
        if representation_size and not distilled:
            self.has_logits = True
            self.num_features = representation_size
            self.pre_logits = nn.Sequential(OrderedDict([
                ("fc", nn.Linear(embed_dim, representation_size)),
                ("act", nn.Tanh())
            ]))
        else:
            self.has_logits = False
            self.pre_logits = nn.Identity()

        # Classifier head(s)
        self.head = nn.Linear(self.num_features, num_classes) if num_classes > 0 else nn.Identity()
        self.head_dist = None
        if distilled:
            self.head_dist = nn.Linear(self.embed_dim, self.num_classes) if num_classes > 0 else nn.Identity()

        # Weight init
        nn.init.trunc_normal_(self.pos_embed, std=0.02)
        if self.dist_token is not None:
            nn.init.trunc_normal_(self.dist_token, std=0.02)

        nn.init.trunc_normal_(self.cls_token, std=0.02)
        self.apply(_init_vit_weights)

    def forward_features(self, x):
        # [B, C, H, W] -> [B, num_patches, embed_dim]
        x = self.patch_embed(x)  # [B, 196, 768]
        # [1, 1, 768] -> [B, 1, 768]
        cls_token = self.cls_token.expand(x.shape[0], -1, -1)
        if self.dist_token is None:
            x = torch.cat((cls_token, x), dim=1)  # [B, 197, 768]
        else:
            x = torch.cat((cls_token, self.dist_token.expand(x.shape[0], -1, -1), x), dim=1)

        x = self.pos_drop(x + self.pos_embed)
        x = self.blocks(x)
        x = self.norm(x)
        if self.dist_token is None:
            return self.pre_logits(x[:, 0])
        else:
            return x[:, 0], x[:, 1]

    def forward(self, x):
        x = self.forward_features(x)
        if self.head_dist is not None:
            x, x_dist = self.head(x[0]), self.head_dist(x[1])
            if self.training and not torch.jit.is_scripting():
                # during inference, return the average of both classifier predictions
                return x, x_dist
            else:
                return (x + x_dist) / 2
        else:
            x = self.head(x)
        return x


 #由于最终分类是在最后关注块中计算的类令牌上进行的，

#输出将不会受到最后一层中的14x14通道的影响。

#输出相对于它们的梯度，将为0！

#我们应该在最后的注意力块之前选择任何一层。
       所以  target_layers = [model.blocks[-1].norm1]



https://blog.csdn.net/qq_37541097/article/details/123089851
https://blog.csdn.net/YI_SHU_JIA/article/details/127223374

In [2]:
class ReshapeTransform:
    def __init__(self, model):
        input_size = model.patch_embed.img_size
        patch_size = model.patch_embed.patch_size
        self.h = input_size[0] // patch_size[0]
        self.w = input_size[1] // patch_size[1]

    def __call__(self, x):
        # remove cls token and reshape
        # [batch_size, num_tokens, token_dim]
        result = x[:, 1:, :].reshape(x.size(0),
                                     self.h,
                                     self.w,
                                     x.size(2))

        # Bring the channels to the first dimension,
        # like in CNNs.
        # [batch_size, H, W, C] -> [batch, C, H, W]
        result = result.permute(0, 3, 1, 2)
        return result

In [3]:
target_layers = [model.blocks[-1].norm1]
data_transform = transforms.Compose([transforms.ToTensor(),
                                         transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
if 1:# load image
    img_path = "/homeB/liangxiaoyu/23w0322/vit_explain/jacobgil_vit_geadcam/grad_cam/both.png"
    assert os.path.exists(img_path), "file: '{}' dose not exist.".format(img_path)
    img = Image.open(img_path).convert('RGB')
    img = np.array(img, dtype=np.uint8)
    img = center_crop_img(img, 224)
    # [C, H, W]
    img_tensor = data_transform(img)
    # expand batch dimension
    # [C, H, W] -> [N, C, H, W]
    input_tensor = torch.unsqueeze(img_tensor, dim=0)    

In [4]:
input_tensor.size()

torch.Size([1, 3, 224, 224])

# 完整

In [ ]:
if 1:
    cam = GradCAM(model=model,
                  target_layers=target_layers,
                  use_cuda=False,
                  reshape_transform=ReshapeTransform(model))
    #target_category = 281  #281  tabby, tabby cat
    # target_category = 254  # pug, pug-dog
    target_category = 313

    grayscale_cam = cam(input_tensor=input_tensor, target_category=target_category)#target_category=target_category

    grayscale_cam = grayscale_cam[0, :]
    visualization = show_cam_on_image(img / 255., grayscale_cam, use_rgb=True)
    plt.imshow(visualization)
    plt.show()

# or拆解

## 实例化init  cam  定义target_category

In [12]:
cam = GradCAM(model=model,
                  target_layers=target_layers,
                  use_cuda=False,
                  reshape_transform=ReshapeTransform(model))

target_category = 281
#target_layers = [model.blocks[-1].norm1]

In [6]:
#see
cam.target_layers#[LayerNorm((768,), eps=1e-05, elementwise_affine=True)]
cam.reshape_transform# <__main__.ReshapeTransform at 0x7f7cdc1011f0>
cam.cuda#False
cam.activations_and_grads#<utils.ActivationsAndGradients at 0x7f7cdc0cfb50>

In [15]:
#see
cam.activations_and_grads.gradients
cam.activations_and_grads.activations
cam.activations_and_grads.reshape_transform#<__main__.ReshapeTransform at 0x7f7cdc1011f0>
cam.activations_and_grads.handles #[<torch.utils.hooks.RemovableHandle at 0x7f7cdc0cf8e0>,<torch.utils.hooks.RemovableHandle at 0x7f7cdc0cf850>]

## 1output = self.activations_and_grads(input_tensor)
注册hook
前向运行
正向传播得到网络输出logits(未经过softmax)

In [13]:
output = cam.activations_and_grads(input_tensor)

## 2if isinstance(target_category, int):

In [8]:
target_category = [target_category] * input_tensor.size(0)

In [18]:
target_category# 281 --- [281]  包装成batch

array([775])

In [14]:
# target_category = None
# if target_category is None:
#     target_category = np.argmax(output.cpu().data.numpy(), axis=-1)
#     print(f"category id: {target_category}")#category id: [775]

category id: [775]


## 3 zerograd lossbackward

In [21]:
#output.size()#torch.Size([1, 1000])
target_category[0]
output[0,775]

#loss 是附复数  倒数下降？

tensor(1.6597, grad_fn=<SelectBackward0>)

In [22]:
cam.model.zero_grad()
loss = cam.get_loss(output, target_category)
loss.backward(retain_graph=True)

## 4cam_per_layer = self.compute_cam_per_layer(input_tensor)

In [25]:
cam_per_layer = cam.compute_cam_per_layer(input_tensor)

In [51]:
len(cam_per_layer)#1
cam_per_layer[0].shape#(1, 1, 224, 224)

(1, 1, 224, 224)

### 4.1 拆开看看

In [38]:
activations_list = [a.cpu().data.numpy()
                            for a in cam.activations_and_grads.activations]#1 * (1, 768, 14, 14)
grads_list = [g.cpu().data.numpy()
                      for g in cam.activations_and_grads.gradients]#1 * (1, 768, 14, 14)
target_size = cam.get_target_width_height(input_tensor)#(224, 224)

### 4.2 cam = self.get_cam_image(layer_activations, layer_grads)

In [57]:
# for layer_activations, layer_grads in zip(activations_list, grads_list):
#     print(layer_activations.shape, layer_grads.shape)
#(1, 768, 14, 14) (1, 768, 14, 14)
layer_activations = activations_list[0]
layer_grads = grads_list[0]

In [62]:
#get_cam_weights###cam = cam.get_cam_image(layer_activations, layer_grads)
#np.mean(layer_grads, axis=(2, 3), keepdims=True).shape#(1, 768, 1, 1)
weights = np.mean(layer_grads, axis=(2, 3), keepdims=True)
weighted_activations = weights * layer_activations#(1, 768, 14, 14)
cam = weighted_activations.sum(axis=1)#(1, 14, 14)

In [65]:
cam[cam < 0] = 0 # 负数全变成0了

这段代码定义了一个名为 scale_cam_image 的函数，用于对 CAM 图像进行缩放和归一化处理。

首先，它初始化了一个空的列表 result 用于存储处理后的图像。

接下来，对于输入的每张 CAM 图像，在每个通道上执行以下操作：

将图像中的像素值减去该图像中的最小值，以确保最小值变为 0。
将图像中的像素值除以（加上一个很小的值，以防止除以零）图像中的最大值，以将像素值缩放到 [0, 1] 范围内。
然后，如果提供了 target_size（目标尺寸），则使用 OpenCV 的 cv2.resize 方法将图像调整为目标尺寸。

最后，将处理后的图像添加到 result 列表中。

最终，将 result 转换为 numpy 数组，并以数据类型 np.float32 返回结果。

总结而言，该函数实现了对 CAM 图像进行最小-最大缩放和归一化的功能。它将图像的最小值转换为 0，将像素值缩放到 [0, 1] 范围内，并可选择将图像调整为指定的目标尺寸。

In [68]:
scaled = GradCAM.scale_cam_image(cam, target_size)#scaled.shape  (1, 224, 224)

scaled[:, None, :]是对numpy数组 scaled 进行索引操作。以下是对索引操作的解释：

在numpy中，用冒号（:）表示整个轴的索引范围，以逗号分隔的数字或冒号（:）表示各个维度的索引。

在这里，scaled[:, None, :] 中的冒号表示对应维度上的所有元素。通过在某个维度上使用冒号表示选择该维度上的所有元素。

具体到这段代码中，scaled[:, None, :] 表示对 scaled 数组进行索引，其中：

第一个冒号 : 表示选择所有的行。
None 表示在行维度上插入一个新的维度，相当于将每一行变成一个单独的子数组。
第二个冒号 : 表示选择所有的列。
因此，scaled[:, None, :] 这个索引操作的结果是将 scaled 数组的每一行按照原样保留，并且在第二个维度上添加了一个新的维度。这样做的目的可能是为了使得每行的数据可以独立处理或者与其他张量进行拼接等操作。

In [71]:
scaled[:, None, :].shape

(1, 1, 224, 224)

## 5return self.aggregate_multi_layers(cam_per_layer)

这段代码是一个函数 aggregate_multi_layers 的实现，其功能是对多层 CAM 进行合并和聚合。

首先，函数接受一个名为 cam_per_target_layer 的参数，它是一个由多个 CAM 矩阵组成的列表。每个 CAM 矩阵表示一个目标层的注意力图。

然后，函数执行以下操作：

使用 np.concatenate 函数将 cam_per_target_layer 列表中的 CAM 矩阵沿着第二个轴（axis=1）进行连接。这将生成一个新的矩阵 cam_per_target_layer，其中每一列代表一个目标层的注意力图。

使用 np.maximum 函数将 cam_per_target_layer 中的每个元素取最大值和0之间的较大值。这样做的目的是去除负值，只保留正值部分。

使用 np.mean 函数计算 cam_per_target_layer 沿着第二个轴（axis=1）的平均值。这将得到一个聚合后的 CAM 数组，其中每个元素表示对应位置的各个目标层的注意力图的平均值。

调用 self.scale_cam_image 方法，传入聚合后的 CAM 数组，并返回经过缩放和归一化处理后的结果。

总结而言，该函数通过将多个目标层的 CAM 图像进行连接、去除负值、求平均以及缩放归一化的操作，实现了对多层 CAM 图像的聚合和处理。最终返回处理后的结果。

In [ ]:
cam.aggregate_multi_layers(cam_per_layer)

00

In [23]:
cam.activations_and_grads.activations[0].shape#torch.Size([1, 768, 14, 14]) 
#activation = self.reshape_transform(activation) 之后再保存的

torch.Size([1, 768, 14, 14])

In [37]:
cam.activations_and_grads.gradients[0].shape#torch.Size([1, 768, 14, 14])

torch.Size([1, 768, 14, 14])

In [24]:
cam.reshape_transform.w

14